# Partitioning Algebraic Graphs
__Jordan Jalving and Victor M. Zavala__ <br>
__University of Wisconsin-Madison__

In [3]:
using Pkg
Pkg.activate("/home/jordan/.julia/dev/ModelGraphs")

"/home/jordan/.julia/dev/ModelGraphs/Project.toml"

In [4]:
using ModelGraphs
using JuMP
using GLPK
using Ipopt

## Create the ModelGraph
__Here we wreate a ModelGraph and add some nodes__

In [5]:
mg = ModelGraph()

n1 = add_node!(mg)
n2 = add_node!(mg)
n3 = add_node!(mg)
n4 = add_node!(mg)

Model Node w/ 0 Variable(s)

#### Previously, we showed how you can define variables and constraints directly on ModelNodes.  We can also use JuMP to create our models and then set them to ModelNodes afterwards.  This is helpful if for instance, we want to set a different model onto an existing ModelNode.


In [6]:
#Set a model on node 1
m1 = Model()
@variable(m1,0 <= x <= 2)
@variable(m1,0 <= y <= 3)
@constraint(m1,x+y <= 4)
@objective(m1,Min,x)

#Set a model on node 2
m2 = Model()
@variable(m2,x >= 1)
@variable(m2,0 <= y <= 5)
@NLconstraint(m2,exp(x)+y <= 7)
@objective(m2,Min,x)

m3 = Model()
@variable(m3,x >= 0)

m4 = Model()
@variable(m4,0 <= x <= 1)


#Set models on nodes and edges
set_model(n1,m1)     #set m1 to node 1.  Updates reference on m1
set_model(n2,m2)
set_model(n3,m3)
set_model(n4,m4)

Model Node w/ 1 Variable(s)

## Add Link Constraints
__Create two link constraints that connect our model nodes.  Underneath, this is actually creating a hypergraph representation of the model (i.e. hyperedges are added when we create link constraints.)__

In [8]:
@linkconstraint(mg,n4[:x] == n1[:x])
@linkconstraint(mg,n1[:y] + n2[:y] + n3[:x] <= 2 )

LinkConstraintRef(Model Graph:
model nodes: 4
link variables: 0
link constraints: 2
, 2, Link edge w/ 1 Constraint(s))

In [10]:
hypergraph = gethypergraph(mg)

Hypergraph: (4 , 2)

## Hypergraph Partitioning
__Since we have a hypergraph, we can perform hypergraph partitioning using the KaHyPar Julia Interface__

In [13]:
using KaHyPar
using SparseArrays

### Minimize the edge cut subject to balance constraints

In [16]:
A = sparse(hypergraph)
partition1 = KaHyPar.partition(A,2,configuration = :edge_cut)


******************************************************************************** 
*                          Top Level Preprocessing..                           * 
******************************************************************************** 
Performing community detection: 
  hypergraph is a graph = false 
  # communities         = 2 
  modularity            = 0.221453 
******************************************************************************** 
*                                Coarsening...                                 * 
******************************************************************************** 
Hypergraph Information 
Name : Coarsened Hypergraph 
Type: edgeWeights=true nodeWeights=true 
# HNs : 4 # HEs : 2 # pins: 5 
HE size             HE weight           HN degree           HN weight 
| min= 2            | min= 1            | min= 1            | min= 1           
| Q1 = 2            | Q1 = 1            | Q1 = 1            | Q1 = 1           
| med= 2            

4-element Array{Int64,1}:
 0
 1
 1
 0

### Minimize connectivity (i.e. communication)

In [17]:
partition2 = KaHyPar.partition(A,2,configuration = :connectivity)


******************************************************************************** 
*                          Top Level Preprocessing..                           * 
******************************************************************************** 
Performing community detection: 
  hypergraph is a graph = false 
  # communities         = 2 
  modularity            = 0.221453 
******************************************************************************** 
*                                Coarsening...                                 * 
******************************************************************************** 
Hypergraph Information 
Name : Coarsened Hypergraph 
Type: edgeWeights=true nodeWeights=true 
# HNs : 4 # HEs : 2 # pins: 5 
HE size             HE weight           HN degree           HN weight 
| min= 2            | min= 1            | min= 1            | min= 1           
| Q1 = 2            | Q1 = 1            | Q1 = 1            | Q1 = 1           
| med= 2            

4-element Array{Int64,1}:
 0
 1
 1
 0

## Graph Partitioning
__We can also perform graph partitioning if we project the hypergraph into a graph space.  We provide three graph projections to do this.__

In [20]:
using Metis
using NestedHyperGraphs

In [25]:
clique_graph,projection_map = clique_expansion(hypergraph)
A2 = sparse(clique_graph.lightgraph)
println(A2)
graph_partition = Metis.partition(A2,2)


  [2, 1]  =  1
  [3, 1]  =  1
  [4, 1]  =  1
  [1, 2]  =  1
  [3, 2]  =  1
  [1, 3]  =  1
  [2, 3]  =  1
  [1, 4]  =  1


4-element Array{Int32,1}:
 1
 1
 1
 1

In [ ]:
# bipartite_graph,projection_map = star_expansion(hypergraph)
# dual_clique_graph,projection_map = dual_clique_expansion(hypergraph)

## Apply Partitioning Results to the ModelGraph
__Use a HyperPartition object to communicate partition information to a ModelGraph.  The idea is that a HyperPartition is a general partitioning interface.  We provide various methods to create HyperPartition objects from common input formats.__

In [26]:
hyperpartition = HyperPartition(hypergraph,partition1);

In [ ]:
#hyperpartition = HyperPartition(hypergraph,graph_partition,projection_map)

__With a HyperPartition, we can aggregate a ModelGraph.  Aggregation is necessary to transform a ModelGraph into a standard block-structured form.__

In [ ]:
aggregated_graph,aggregation_map = aggregate(mg,hyperpartition)

## Using ModelGraph Solvers

In [ ]:
using ModelGraphSolvers 
using Distributed

dual_decomposition_optimizer = DDOptimizer(workers())
optimize!(aggregated_graph,dual_decomposition_optimizer)